In [18]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

import pandas as pd

In [41]:
oai_auth = '2e5b9946f66648789d1a0cc12f01f046'
oai_endpoint = 'https://me-m1tege32-eastus2.openai.azure.com/'

In [45]:
llm = AzureChatOpenAI(
    azure_endpoint=oai_endpoint,
    openai_api_key = oai_auth,
    azure_deployment='gpt-35-turbo',
    openai_api_version="2024-05-01-preview",
)

client = AzureOpenAI(
  api_key = "4098e32d0091462c930c526f29a4f1f0",  
  api_version = "2023-05-15",
  azure_deployment='text-embedding-ada-002',
  azure_endpoint = "https://gmarropenai.openai.azure.com/"
)

folder_template = "../transcript/what’s_wrong_with_donald_trump.txt"

# Open the file in read mode ('r')
with open(folder_template, 'r') as file:
    content = file.read()


In [36]:
def refine_summary(transcript_path:str=None):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=10)

    cook = TextLoader(transcript_path, encoding = 'UTF-8').load()
    text = text_splitter.split_documents(cook)

    return text

ll = refine_summary(transcript_path=folder_template)

In [37]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills = pd.DataFrame({'text':[i.page_content for i in ll]})
df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [38]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]

In [46]:
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [48]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "What makes Donald second term more dangerous?", top_n=4)

,text,n_tokens,ada_v2,similarities
23,to do what Mike Pence would not. Elon Musk is ...,309,"[-0.026767728850245476, -0.017276007682085037,...",0.827407
11,to tell you. Do you agree with that? He's a wa...,338,"[-0.023426249623298645, -0.00290662725456059, ...",0.823870
18,Nikki Haley and I fought. And I beat her by 50...,327,"[-0.03176213428378105, -0.01694893091917038, 0...",0.823530
14,and I were so concerned because we didn't want...,312,"[-0.019278954714536667, -0.028694888576865196,...",0.818462


In [67]:
allTEXTS = res['text'].to_list() 
single_string = " ".join(allTEXTS)

with open("../answers/wow.txt", "w", encoding="utf-8") as file:
    file.write(single_string)

In [69]:
def refine_summary(transcript_path:str=None):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=10)

    cook = TextLoader(transcript_path, encoding = 'UTF-8').load()
    text = text_splitter.split_documents(cook)

    prompt = """
            Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

    question_prompt = PromptTemplate(
        template=prompt, input_variables=["text"]
    )

    refine_prompt_template = """
                Write a concise summary of the following text delimited by triple backquotes.
                Return your response in bullet points which covers the key points of the text.
                IF IT SOUNDS LIKE AN ADVERTISMENT RETURN 'AD NO RELEVANT'
                ```{text}```
                BULLET POINT SUMMARY:
                """

    refine_template = PromptTemplate(
        template=refine_prompt_template, input_variables=["text"])

    # Load refine chain
    chain = load_summarize_chain(
        llm=llm,
        chain_type="refine",
        question_prompt=question_prompt,
        refine_prompt=refine_template,
        return_intermediate_steps=True,
        input_key="input_documents",
        output_key="output_text",
    )

    result_summary = chain({"input_documents": text}, return_only_outputs=False)
    answer = result_summary['output_text']

    # Open the file in write mode and save the transcription
    with open("../answers/wow.txt", "w", encoding="utf-8") as file:
        file.write(answer)

    return answer

aah=refine_summary(transcript_path='\../answers/wow.txt')

/var/folders/08/chgp4vr54wzb4d1lbl1tqls40000gn/T/ipykernel_10167/852053879.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result_summary = chain({"input_documents": text}, return_only_outputs=False)


In [70]:
aah

"- FEMA officials were concerned about cutting off money to people whose homes were burning down at the direction of the president\n- Trump administration was known for disregarding the president's orders and just doing their jobs\n- Trump repeatedly proposed extreme measures such as firing patriot missiles at suspected drug labs in Mexico and launching nuclear weapons\n- Trump talked about using government machinery against political enemies, pulling out of NATO, and using untested treatments for COVID\n- During George Floyd protests, Trump wanted to use military force against protesters"